# Задача 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score, coverage_error, label_ranking_loss, mean_squared_error
#classificators
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import collections

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
scaler = StandardScaler()
label = LabelEncoder()

In [ ]:
df=pd.read_excel('train.xlsx')
test=pd.read_excel('test.xlsx')

In [ ]:
df2=df.copy()

Генерим немного фич

In [ ]:
df.Polis=df.POLICY_END_MONTH-df.POLICY_BEGIN_MONTH

In [ ]:
df.head()

In [ ]:
df.columns

In [83]:
df.dtypes

POLICY_ID                          int64
POLICY_BEGIN_MONTH                 int64
POLICY_END_MONTH                   int64
POLICY_SALES_CHANNEL               int64
POLICY_SALES_CHANNEL_GROUP         int64
POLICY_BRANCH                     object
POLICY_MIN_AGE                     int64
POLICY_MIN_DRIVING_EXPERIENCE      int64
VEHICLE_MAKE                       int64
VEHICLE_MODEL                     object
VEHICLE_ENGINE_POWER             float64
VEHICLE_IN_CREDIT                  int64
VEHICLE_SUM_INSURED              float64
POLICY_INTERMEDIARY               object
INSURER_GENDER                     int64
POLICY_CLM_N                      object
POLICY_CLM_GLT_N                  object
POLICY_PRV_CLM_N                  object
POLICY_PRV_CLM_GLT_N              object
CLIENT_HAS_DAGO                    int64
CLIENT_HAS_OSAGO                   int64
POLICY_COURT_SIGN                  int64
CLAIM_AVG_ACC_ST_PRD             float64
POLICY_HAS_COMPLAINTS              int64
POLICY_YEARS_REN

In [ ]:
df.columns

In [ ]:
df.isnull().values.any()

In [85]:
df.POLICY_CLM_N

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8        1S
9         0
10       1S
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29       1S
         ..
77377     0
77378     0
77379     0
77380    1S
77381     0
77382     0
77383     0
77384    1S
77385    1S
77386     0
77387     0
77388    1L
77389    1S
77390     0
77391     0
77392     0
77393     0
77394    1S
77395     0
77396     2
77397     0
77398     0
77399     0
77400     0
77401     0
77402     0
77403     0
77404    1S
77405     0
77406     0
Name: POLICY_CLM_N, dtype: object

In [84]:
label.fit_transform(df['POLICY_CLM_N'])

TypeError: unorderable types: int() > str()

Предобработка:категориальное и частотное кодирование, скалер

In [ ]:
def preproc(data):
    cat_val=['VEHICLE_MAKE',   'INSURER_GENDER']
    for i in cat_val:
        df[i]=label.fit_transform(df[i])
    freq_val=['VEHICLE_MODEL', 'POLICY_CLM_N'    'POLICY_INTERMEDIARY', 'CLIENT_REGISTRATION_REGION','POLICY_BRANCH','POLICY_PRV_CLM_GLT_N','POLICY_PRV_CLM_N','POLICY_CLM_GLT_N',]
    for j in freq_val:
        cnt=collections.Counter(df[j])
        for i in range(len(cnt)):
            df[j][df[j]==list(cnt.keys())[i]]=list(cnt.values())[i]
    X=scaler.fit_transform(data)
    return X

In [ ]:
df.POLICY_IS_RENEWED.value_counts()

In [82]:
Y=df.POLICY_IS_RENEWED
df=df.drop('POLICY_IS_RENEWED',axis=1)
X=preproc(df)

KeyError: 'POLICY_CLM_NPOLICY_INTERMEDIARY'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0)

In [ ]:
clf = RandomForestClassifier(random_state=0,class_weight="balanced")
tuned_parameters = [{'min_samples_split':np.arange(1, 4, 1)},
                     {'n_estimators':np.arange(5, 25, 5)},
                      {'max_depth':np.arange(1, 6, 1)}]
gs = GridSearchCV(clf, tuned_parameters, cv=8,scoring="f1_weighted" ,verbose=True)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_

In [ ]:
preds_train=pd.Series(gs.predict(X_train))
preds_train.value_counts()[:15]

In [ ]:
print ("MSE: ", mean_squared_error(y_train, preds_train))
print ("F-мера: ", f1_score(y_train, preds_train))

Смотрим на валидацию

In [ ]:
print ("MSE: ",mean_squared_error(y_test, gs.predict(X_test)))
print ("F-мера: ",f1_score(y_test, gs.predict(X_test)))

# TEST

In [ ]:
test=preproc(test)

In [ ]:
ans=gs.predict(test)
ans.value_counts()[:15]

In [ ]:
ans=pd.concat([test2["Node"],ans], axis=1)
ans.to_csv("answer.csv")